# 本地模型工具调用实现

这个 notebook 演示了如何在不使用 OpenAI 封装的情况下，基于本地模型实现工具调用功能。
我们将从零开始构建一个完整的工具调用系统。

## 核心原理

工具调用的核心原理是：
1. 通过 prompt 告诉模型有哪些工具可用
2. 模型生成结构化的工具调用请求（通常是 JSON 格式）
3. 解析模型输出，提取工具调用信息
4. 执行相应的工具函数
5. 将工具执行结果返回给模型继续对话

In [19]:
import json
import re
import asyncio
from datetime import datetime
from typing import Dict, List, Any, Optional, Callable
from dataclasses import dataclass
import math

import os
from dotenv import load_dotenv
from fallback_openai_client import AsyncFallbackOpenAIClient

load_dotenv()

DEEPSEEK_API_KEY = os.getenv("API_KEY")
DEEPSEEK_BASE_URL = os.getenv("BASE_URL")
DEEPSEEK_MODEL = "deepseek-chat"

print("🚀 本地工具调用实现初始化完成")

🚀 本地工具调用实现初始化完成


## 1. 定义工具函数和描述

首先定义我们要提供给模型的工具函数。

In [20]:
# 工具函数定义
def get_current_time() -> str:
    """获取当前时间"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate_area(shape: str, **kwargs) -> dict:
    """计算几何图形面积"""
    if shape == "circle":
        radius = kwargs.get("radius", 0)
        if radius <= 0:
            return {"error": "圆的半径必须大于0"}
        return {"shape": "circle", "area": math.pi * radius ** 2}
    elif shape == "rectangle":
        width = kwargs.get("width", 0)
        height = kwargs.get("height", 0)
        if width <= 0 or height <= 0:
            return {"error": "矩形的宽度和高度必须大于0"}
        return {"shape": "rectangle", "area": width * height}
    else:
        return {"error": f"不支持的图形类型: {shape}"}

def get_weather(city: str) -> str:
    """获取城市天气信息（模拟）"""
    weather_data = {
        "北京": "晴天，气温 15°C，微风",
        "上海": "多云，气温 18°C，东南风",
        "广州": "雨天，气温 22°C，南风",
        "深圳": "晴天，气温 25°C，无风",
        "杭州": "阴天，气温 16°C，西北风"
    }
    return weather_data.get(city, f"抱歉，暂时无法获取 {city} 的天气信息")

def search_knowledge(query: str) -> str:
    """搜索知识库（模拟）"""
    knowledge_base = {
        "python": "Python 是一种解释型、面向对象、动态数据类型的高级程序设计语言。",
        "机器学习": "机器学习是人工智能的一个分支，让计算机通过数据学习模式。",
        "深度学习": "深度学习是机器学习的子集，使用神经网络模拟人脑学习过程。",
        "ai": "人工智能（AI）是计算机科学的一个领域，致力于创建智能机器。"
    }
    
    query_lower = query.lower()
    for key, value in knowledge_base.items():
        if key in query_lower:
            return value
    
    return f"抱歉，没有找到关于 '{query}' 的相关信息"

print("✅ 工具函数定义完成")

✅ 工具函数定义完成


## 2. 工具描述和注册系统

创建一个工具注册系统，用于管理工具的描述和调用。

In [21]:
@dataclass
class ToolParameter:
    """工具参数定义"""
    name: str
    type: str
    description: str
    required: bool = True

@dataclass
class ToolDefinition:
    """工具定义"""
    name: str
    function: Callable
    description: str
    parameters: List[ToolParameter]

class LocalToolRegistry:
    """本地工具注册表"""
    
    def __init__(self):
        self.tools: Dict[str, ToolDefinition] = {}
    
    def register_tool(self, tool_def: ToolDefinition):
        """注册工具"""
        self.tools[tool_def.name] = tool_def
        print(f"✅ 注册工具: {tool_def.name}")
    
    def get_tool(self, name: str) -> Optional[ToolDefinition]:
        """获取工具定义"""
        return self.tools.get(name)
    
    def list_tools(self) -> List[str]:
        """列出所有工具名称"""
        return list(self.tools.keys())
    
    def generate_tool_prompt(self) -> str:
        """生成工具描述的 prompt"""
        if not self.tools:
            return "当前没有可用的工具。"
        
        prompt = "你有以下工具可以使用：\n\n"
        
        for tool_name, tool_def in self.tools.items():
            prompt += f"**{tool_name}**: {tool_def.description}\n"
            prompt += "参数:\n"
            
            if not tool_def.parameters:
                prompt += "  - 无需参数\n"
            else:
                for param in tool_def.parameters:
                    required_str = "必需" if param.required else "可选"
                    prompt += f"  - {param.name} ({param.type}, {required_str}): {param.description}\n"
            prompt += "\n"
        
        prompt += """当你需要调用工具时，请严格按照以下JSON格式输出：

```json
{
  "tool_call": {
    "name": "工具名称",
    "arguments": {
      "参数名": "参数值"
    }
  }
}
```

如果不需要调用工具，请直接回答问题。"""
        
        return prompt

# 创建工具注册表并注册工具
tool_registry = LocalToolRegistry()

# 注册工具
tool_registry.register_tool(ToolDefinition(
    name="get_current_time",
    function=get_current_time,
    description="获取当前的日期和时间",
    parameters=[]
))

tool_registry.register_tool(ToolDefinition(
    name="calculate_area",
    function=calculate_area,
    description="计算几何图形的面积（支持圆形和矩形）",
    parameters=[
        ToolParameter("shape", "string", "图形类型，可选值：circle（圆形）、rectangle（矩形）"),
        ToolParameter("radius", "number", "圆的半径（当shape为circle时必需）", required=False),
        ToolParameter("width", "number", "矩形的宽度（当shape为rectangle时必需）", required=False),
        ToolParameter("height", "number", "矩形的高度（当shape为rectangle时必需）", required=False),
    ]
))

tool_registry.register_tool(ToolDefinition(
    name="get_weather",
    function=get_weather,
    description="获取指定城市的天气信息",
    parameters=[
        ToolParameter("city", "string", "城市名称，如：北京、上海、广州、深圳、杭州")
    ]
))

tool_registry.register_tool(ToolDefinition(
    name="search_knowledge",
    function=search_knowledge,
    description="在知识库中搜索相关信息",
    parameters=[
        ToolParameter("query", "string", "搜索查询词")
    ]
))

print(f"\n📋 已注册 {len(tool_registry.tools)} 个工具")
print("工具列表:", tool_registry.list_tools())

✅ 注册工具: get_current_time
✅ 注册工具: calculate_area
✅ 注册工具: get_weather
✅ 注册工具: search_knowledge

📋 已注册 4 个工具
工具列表: ['get_current_time', 'calculate_area', 'get_weather', 'search_knowledge']


## 3. 工具调用解析器

创建解析器来从模型输出中提取工具调用请求。

In [22]:
class ToolCallParser:
    """工具调用解析器"""
    
    @staticmethod
    def extract_json_from_text(text: str) -> Optional[dict]:
        """从文本中提取JSON"""
        # 尝试多种模式匹配JSON
        patterns = [
            r'```json\s*(\{.*?\})\s*```',  # ```json ... ```
            r'```\s*(\{.*?\})\s*```',      # ``` ... ```
            r'\{[^{}]*"tool_call"[^{}]*\{[^{}]*\}[^{}]*\}',  # 直接匹配tool_call结构
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
            for match in matches:
                try:
                    return json.loads(match)
                except json.JSONDecodeError:
                    continue
        
        # 尝试直接解析整个文本
        try:
            return json.loads(text.strip())
        except json.JSONDecodeError:
            pass
        
        return None
    
    @staticmethod
    def parse_tool_call(text: str) -> Optional[dict]:
        """解析工具调用"""
        json_data = ToolCallParser.extract_json_from_text(text)
        
        if not json_data:
            return None
        
        # 检查是否包含tool_call结构
        if "tool_call" in json_data:
            tool_call = json_data["tool_call"]
            if "name" in tool_call:
                return {
                    "name": tool_call["name"],
                    "arguments": tool_call.get("arguments", {})
                }
        
        # 检查是否直接是工具调用格式
        if "name" in json_data and "arguments" in json_data:
            return {
                "name": json_data["name"],
                "arguments": json_data.get("arguments", {})
            }
        
        return None

# 测试解析器
parser = ToolCallParser()

test_cases = [
    '```json\n{"tool_call": {"name": "get_current_time", "arguments": {}}}\n```',
    '{"tool_call": {"name": "get_weather", "arguments": {"city": "北京"}}}',
    '我需要调用工具：```json\n{"tool_call": {"name": "calculate_area", "arguments": {"shape": "circle", "radius": 5}}}\n```',
    '普通回答，不需要调用工具'
]

print("🧪 测试工具调用解析器:")
for i, case in enumerate(test_cases, 1):
    result = parser.parse_tool_call(case)
    print(f"测试 {i}: {'✅ 解析成功' if result else '❌ 无工具调用'}")
    if result:
        print(f"   工具: {result['name']}, 参数: {result['arguments']}")

🧪 测试工具调用解析器:
测试 1: ✅ 解析成功
   工具: get_current_time, 参数: {}
测试 2: ✅ 解析成功
   工具: get_weather, 参数: {'city': '北京'}
测试 3: ✅ 解析成功
   工具: calculate_area, 参数: {'shape': 'circle', 'radius': 5}
测试 4: ❌ 无工具调用


## 4. 本地工具调用执行器

创建工具调用的执行器。

In [23]:
class LocalToolExecutor:
    """本地工具执行器"""
    
    def __init__(self, tool_registry: LocalToolRegistry):
        self.tool_registry = tool_registry
    
    async def execute_tool(self, tool_name: str, arguments: dict) -> dict:
        """执行工具调用"""
        try:
            tool_def = self.tool_registry.get_tool(tool_name)
            if not tool_def:
                return {
                    "success": False,
                    "error": f"工具 '{tool_name}' 不存在",
                    "available_tools": self.tool_registry.list_tools()
                }
            
            # 验证必需参数
            missing_params = []
            for param in tool_def.parameters:
                if param.required and param.name not in arguments:
                    missing_params.append(param.name)
            
            if missing_params:
                return {
                    "success": False,
                    "error": f"缺少必需参数: {', '.join(missing_params)}"
                }
            
            # 执行工具函数
            if arguments:
                result = tool_def.function(**arguments)
            else:
                result = tool_def.function()
            
            return {
                "success": True,
                "tool_name": tool_name,
                "arguments": arguments,
                "result": result
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": f"工具执行失败: {str(e)}",
                "tool_name": tool_name,
                "arguments": arguments
            }

# 创建执行器
executor = LocalToolExecutor(tool_registry)

# 测试执行器
print("🧪 测试工具执行器:")

test_executions = [
    ("get_current_time", {}),
    ("calculate_area", {"shape": "circle", "radius": 5}),
    ("get_weather", {"city": "北京"}),
    ("search_knowledge", {"query": "Python"}),
    ("nonexistent_tool", {})  # 测试不存在的工具
]

for tool_name, args in test_executions:
    result = await executor.execute_tool(tool_name, args)
    status = "✅" if result["success"] else "❌"
    print(f"{status} {tool_name}: {result}")

🧪 测试工具执行器:
✅ get_current_time: {'success': True, 'tool_name': 'get_current_time', 'arguments': {}, 'result': '2026-01-14 16:10:49'}
✅ calculate_area: {'success': True, 'tool_name': 'calculate_area', 'arguments': {'shape': 'circle', 'radius': 5}, 'result': {'shape': 'circle', 'area': 78.53981633974483}}
✅ get_weather: {'success': True, 'tool_name': 'get_weather', 'arguments': {'city': '北京'}, 'result': '晴天，气温 15°C，微风'}
✅ search_knowledge: {'success': True, 'tool_name': 'search_knowledge', 'arguments': {'query': 'Python'}, 'result': 'Python 是一种解释型、面向对象、动态数据类型的高级程序设计语言。'}
❌ nonexistent_tool: {'success': False, 'error': "工具 'nonexistent_tool' 不存在", 'available_tools': ['get_current_time', 'calculate_area', 'get_weather', 'search_knowledge']}


## 5. 本地工具调用主类

整合所有组件，创建完整的本地工具调用系统。

In [24]:
class LocalToolUseChat:
    """本地工具调用聊天系统"""
    
    def __init__(self, api_key: str, base_url: str, model: str, tool_registry: LocalToolRegistry):
        self.client = AsyncFallbackOpenAIClient(
            primary_api_key=api_key,
            primary_base_url=base_url,
            primary_model_name=model
        )
        self.tool_registry = tool_registry
        self.executor = LocalToolExecutor(tool_registry)
        self.parser = ToolCallParser()
    
    def build_system_prompt(self) -> str:
        """构建系统提示词"""
        return f"""你是一个智能助手，可以使用以下工具来帮助用户：

{self.tool_registry.generate_tool_prompt()}

请注意：
1. 只有在确实需要调用工具时才使用工具
2. 严格按照JSON格式输出工具调用
3. 如果不需要工具，直接回答问题
4. 每次只调用一个工具
"""
    
    async def chat(self, user_message: str, conversation_history: List[dict] = None) -> dict:
        """进行对话，支持工具调用"""
        if conversation_history is None:
            conversation_history = []
        
        # 构建消息
        messages = [
            {"role": "system", "content": self.build_system_prompt()}
        ]
        messages.extend(conversation_history)
        messages.append({"role": "user", "content": user_message})
        
        max_tool_calls = 3  # 防止无限循环
        tool_call_count = 0
        
        while tool_call_count < max_tool_calls:
            try:
                # 调用模型
                response = await self.client.chat_completions_create(
                    messages=messages,
                    max_tokens=1000,
                    temperature=0.7
                )
                
                assistant_response = response.choices[0].message.content
                
                # 尝试解析工具调用
                tool_call = self.parser.parse_tool_call(assistant_response)
                
                if tool_call:
                    print(f"🔧 检测到工具调用: {tool_call['name']}")
                    print(f"📝 参数: {tool_call['arguments']}")
                    
                    # 执行工具
                    tool_result = await self.executor.execute_tool(
                        tool_call['name'], 
                        tool_call['arguments']
                    )
                    
                    print(f"✅ 工具执行结果: {tool_result}")
                    
                    # 将工具调用和结果添加到对话历史
                    messages.append({
                        "role": "user", 
                        "content": f"工具调用结果: {json.dumps(tool_result, ensure_ascii=False)}"
                    })
                    
                    tool_call_count += 1
                    continue  # 继续下一轮对话
                else:
                    # 没有工具调用，返回最终响应
                    return {
                        "response": assistant_response,
                        "tool_calls_used": tool_call_count,
                        "conversation": messages,
                        "success": True
                    }
                    
            except Exception as e:
                return {
                    "error": f"对话失败: {str(e)}",
                    "tool_calls_used": tool_call_count,
                    "conversation": messages,
                    "success": False
                }
        
        # 达到最大工具调用次数
        return {
            "response": "已达到最大工具调用次数限制",
            "tool_calls_used": tool_call_count,
            "conversation": messages,
            "success": True
        }
    
    async def close(self):
        """关闭客户端"""
        await self.client.close()

# 创建本地工具调用聊天系统
local_chat = LocalToolUseChat(
    api_key=DEEPSEEK_API_KEY,
    base_url=DEEPSEEK_BASE_URL,
    model=DEEPSEEK_MODEL,
    tool_registry=tool_registry
)

print("✅ 本地工具调用聊天系统初始化完成")

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
✅ 本地工具调用聊天系统初始化完成


## 6. 测试本地工具调用系统

测试我们的本地工具调用实现。

In [25]:
async def test_local_tool_use():
    """测试本地工具调用系统"""
    
    test_cases = [
        "你好，你能做什么？",
        "现在几点了？",
        "帮我计算半径为8的圆的面积",
        "北京今天天气怎么样？",
        "搜索一下关于机器学习的信息",
        "先告诉我现在时间，然后查询上海的天气",
        "计算一个长5宽3的矩形面积"
    ]
    
    print("🚀 开始测试本地工具调用系统")
    print("=" * 70)
    
    for i, question in enumerate(test_cases, 1):
        print(f"\n📋 测试 {i}: {question}")
        print("-" * 50)
        
        try:
            result = await local_chat.chat(question)
            
            if result["success"]:
                print(f"🤖 助手回复: {result['response']}")
                print(f"📊 使用工具次数: {result['tool_calls_used']}")
            else:
                print(f"❌ 错误: {result.get('error', '未知错误')}")
                
        except Exception as e:
            print(f"❌ 测试失败: {e}")
        
        print("-" * 50)
    
    await local_chat.close()
    print("\n✅ 测试完成")

# 运行测试
await test_local_tool_use()

🚀 开始测试本地工具调用系统

📋 测试 1: 你好，你能做什么？
--------------------------------------------------


🤖 助手回复: 你好！我可以帮助你完成以下任务：

1. **获取当前时间** - 我可以告诉你当前的日期和时间
2. **计算几何图形的面积** - 我可以计算圆形或矩形的面积
3. **获取天气信息** - 我可以查询指定城市的天气情况
4. **搜索知识库** - 我可以在知识库中搜索相关信息

如果你需要我执行这些任务中的任何一个，请告诉我具体需求。例如：
- "现在几点了？"
- "帮我计算一个半径为5的圆的面积"
- "北京天气怎么样？"
- "搜索关于人工智能的信息"

有什么我可以帮助你的吗？
📊 使用工具次数: 0
--------------------------------------------------

📋 测试 2: 现在几点了？
--------------------------------------------------
🔧 检测到工具调用: get_current_time
📝 参数: {}
✅ 工具执行结果: {'success': True, 'tool_name': 'get_current_time', 'arguments': {}, 'result': '2026-01-14 16:10:56'}
🤖 助手回复: 现在是2026年1月14日，下午4点10分56秒。
📊 使用工具次数: 1
--------------------------------------------------

📋 测试 3: 帮我计算半径为8的圆的面积
--------------------------------------------------
🔧 检测到工具调用: calculate_area
📝 参数: {'shape': 'circle', 'radius': 8}
✅ 工具执行结果: {'success': True, 'tool_name': 'calculate_area', 'arguments': {'shape': 'circle', 'radius': 8}, 'result': {'shape': 'circle', 'area': 201.06192982974676}}
🤖 助手回复: 圆的面积计算公式是 π × r²，其中 r 是半径。

根据计算结果，半径为 8 的圆的面积约为 **201.

## 7. 进阶功能：支持多轮对话

演示如何在多轮对话中保持工具调用的上下文。

In [26]:
async def test_multi_turn_conversation():
    """测试多轮对话中的工具调用"""
    
    local_chat_multi = LocalToolUseChat(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL,
        model=DEEPSEEK_MODEL,
        tool_registry=tool_registry
    )
    
    conversation_history = []
    
    dialogue = [
        "你好，我想了解一些信息",
        "现在几点了？",
        "那北京的天气怎么样？",
        "帮我计算一个半径为10的圆的面积",
        "如果这个圆的半径变成15，面积会是多少？",
        "搜索一下深度学习的相关信息"
    ]
    
    print("🗣️  开始多轮对话测试")
    print("=" * 70)
    
    try:
        for i, user_input in enumerate(dialogue, 1):
            print(f"\n👤 用户 (第{i}轮): {user_input}")
            
            result = await local_chat_multi.chat(user_input, conversation_history)
            
            if result["success"]:
                print(f"🤖 助手: {result['response']}")
                
                # 更新对话历史
                conversation_history.append({"role": "user", "content": user_input})
                conversation_history.append({"role": "assistant", "content": result['response']})
                
                if result['tool_calls_used'] > 0:
                    print(f"🔧 (本轮使用了 {result['tool_calls_used']} 次工具调用)")
            else:
                print(f"❌ 错误: {result.get('error', '未知错误')}")
                break
    
    finally:
        await local_chat_multi.close()
    
    print(f"\n✅ 多轮对话测试完成，共进行了 {len(dialogue)} 轮对话")

# 运行多轮对话测试
await test_multi_turn_conversation()

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
🗣️  开始多轮对话测试

👤 用户 (第1轮): 你好，我想了解一些信息
🤖 助手: 你好！我很乐意帮助你了解信息。请问你想了解哪方面的信息呢？比如：

1. 当前时间
2. 几何图形的面积计算
3. 某个城市的天气情况
4. 其他知识库中的信息

请告诉我你的具体需求，我会尽力帮助你！

👤 用户 (第2轮): 现在几点了？
🔧 检测到工具调用: get_current_time
📝 参数: {}
✅ 工具执行结果: {'success': True, 'tool_name': 'get_current_time', 'arguments': {}, 'result': '2026-01-14 16:11:30'}
🤖 助手: 现在是2026年1月14日，下午4点11分30秒。
🔧 (本轮使用了 1 次工具调用)

👤 用户 (第3轮): 那北京的天气怎么样？
🔧 检测到工具调用: get_weather
📝 参数: {'city': '北京'}
✅ 工具执行结果: {'success': True, 'tool_name': 'get_weather', 'arguments': {'city': '北京'}, 'result': '晴天，气温 15°C，微风'}
🤖 助手: 根据查询结果，北京现在的天气情况是：
- **天气状况**：晴天 ☀️
- **温度**：15°C
- **风力**：微风

天气不错，适合外出活动！
🔧 (本轮使用了 1 次工具调用)

👤 用户 (第4轮): 帮我计算一个半径为10的圆的面积
🔧 检测到工具调用: calculate_area
📝 参数: {'shape': 'circle', 'radius': 10}
✅ 工具执行结果: {'success': True, 'tool_name': 'calculate_area', 'arguments': {'shape': 'circle', 'radius': 10}, 'result': {'shape': 'circle', 'area': 314.1592653589793}}
🤖 助手: 半径为10的圆的面积是 **314.16**（约等于314.16，精确值为314.15926

## 8. 自定义工具扩展示例

演示如何轻松添加新的自定义工具。

In [27]:
# 定义新的自定义工具
def generate_password(length: int = 12, include_symbols: bool = True) -> str:
    """生成随机密码"""
    import random
    import string
    
    if length < 4:
        return "密码长度至少为4位"
    
    chars = string.ascii_letters + string.digits
    if include_symbols:
        chars += "!@#$%^&*"
    
    password = ''.join(random.choice(chars) for _ in range(length))
    return password

def calculate_bmi(weight: float, height: float) -> dict:
    """计算BMI指数"""
    if weight <= 0 or height <= 0:
        return {"error": "体重和身高必须大于0"}
    
    bmi = weight / (height ** 2)
    
    if bmi < 18.5:
        category = "偏瘦"
    elif bmi < 24:
        category = "正常"
    elif bmi < 28:
        category = "偏胖"
    else:
        category = "肥胖"
    
    return {
        "bmi": round(bmi, 2),
        "category": category,
        "weight": weight,
        "height": height
    }

# 创建新的工具注册表并添加扩展工具
extended_registry = LocalToolRegistry()

# 注册原有工具
for tool_name, tool_def in tool_registry.tools.items():
    extended_registry.register_tool(tool_def)

# 注册新工具
extended_registry.register_tool(ToolDefinition(
    name="generate_password",
    function=generate_password,
    description="生成随机密码",
    parameters=[
        ToolParameter("length", "number", "密码长度，默认12位", required=False),
        ToolParameter("include_symbols", "boolean", "是否包含特殊符号，默认true", required=False)
    ]
))

extended_registry.register_tool(ToolDefinition(
    name="calculate_bmi",
    function=calculate_bmi,
    description="计算BMI指数（体重指数）",
    parameters=[
        ToolParameter("weight", "number", "体重（公斤）"),
        ToolParameter("height", "number", "身高（米）")
    ]
))

print(f"✅ 扩展工具注册表创建完成，共有 {len(extended_registry.tools)} 个工具")

# 测试扩展工具
async def test_extended_tools():
    """测试扩展工具"""
    
    extended_chat = LocalToolUseChat(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL,
        model=DEEPSEEK_MODEL,
        tool_registry=extended_registry
    )
    
    test_cases = [
        "帮我生成一个16位的密码",
        "我的体重是70公斤，身高1.75米，帮我计算BMI",
        "生成一个8位不包含特殊符号的密码"
    ]
    
    print("🔧 测试扩展工具功能")
    print("=" * 50)
    
    try:
        for i, question in enumerate(test_cases, 1):
            print(f"\n📋 测试 {i}: {question}")
            
            result = await extended_chat.chat(question)
            
            if result["success"]:
                print(f"🤖 回复: {result['response']}")
                print(f"📊 工具调用次数: {result['tool_calls_used']}")
            else:
                print(f"❌ 错误: {result.get('error')}")
    
    finally:
        await extended_chat.close()
    
    print("\n✅ 扩展工具测试完成")

# 运行扩展工具测试
await test_extended_tools()

✅ 注册工具: get_current_time
✅ 注册工具: calculate_area
✅ 注册工具: get_weather
✅ 注册工具: search_knowledge
✅ 注册工具: generate_password
✅ 注册工具: calculate_bmi
✅ 扩展工具注册表创建完成，共有 6 个工具
⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
🔧 测试扩展工具功能

📋 测试 1: 帮我生成一个16位的密码
🔧 检测到工具调用: generate_password
📝 参数: {'length': 16}
✅ 工具执行结果: {'success': True, 'tool_name': 'generate_password', 'arguments': {'length': 16}, 'result': 'X6DzHjb45i5DSN^a'}
🤖 回复: 这是为您生成的16位随机密码：**X6DzHjb45i5DSN^a**

这个密码包含了大小写字母、数字和特殊符号，安全性较高。请注意妥善保管您的密码，不要与他人分享。
📊 工具调用次数: 1

📋 测试 2: 我的体重是70公斤，身高1.75米，帮我计算BMI
🔧 检测到工具调用: calculate_bmi
📝 参数: {'weight': 70, 'height': 1.75}
✅ 工具执行结果: {'success': True, 'tool_name': 'calculate_bmi', 'arguments': {'weight': 70, 'height': 1.75}, 'result': {'bmi': 22.86, 'category': '正常', 'weight': 70, 'height': 1.75}}
🤖 回复: 根据计算结果，您的BMI指数为22.86。

**解读：**
- **BMI值：** 22.86
- **体重状况：** 正常
- **参考标准：**
  - 偏瘦：< 18.5
  - 正常：18.5 - 23.9
  - 超重：24 - 27.9
  - 肥胖：≥ 28

您的BMI值在正常范围内，说明您的体重相对于身高是健康的。继续保持均衡饮食和适量运动即可。
📊 工具调用次数: 1

📋 测试 3: 

## 9. 总结与对比

### 本地工具调用实现 vs OpenAI 封装实现

| 特性 | 本地实现 | OpenAI 封装 |
|------|----------|-------------|
| **控制程度** | 完全控制 | 依赖 API 规范 |
| **自定义能力** | 高度灵活 | 相对受限 |
| **实现复杂度** | 较复杂 | 简单 |
| **调试难度** | 容易调试 | 黑盒调试 |
| **扩展性** | 容易扩展 | 需要遵循规范 |
| **错误处理** | 自定义处理 | 标准化处理 |

### 本地实现的优势：

1. **完全控制**: 可以自定义工具调用的每个环节
2. **灵活的格式**: 不必严格遵循 OpenAI 的 JSON Schema
3. **自定义解析**: 可以处理模型输出的各种格式
4. **扩展性强**: 容易添加新功能和工具
5. **调试友好**: 可以在每个步骤添加日志和调试信息

### 本地实现的核心组件：

1. **工具注册系统**: 管理可用工具及其描述
2. **Prompt 生成器**: 告诉模型有哪些工具可用
3. **解析器**: 从模型输出中提取工具调用请求
4. **执行器**: 执行实际的工具函数
5. **对话管理**: 处理多轮对话和上下文

### 使用场景：

- **需要高度自定义**: 特殊的工具调用格式或流程
- **本地部署**: 完全离线的工具调用系统
- **复杂工具**: 需要特殊参数验证或处理逻辑
- **调试和开发**: 需要深入了解工具调用的内部机制

这个实现展示了工具调用的底层原理，帮助你理解 AI 模型如何与外部工具交互！🚀